In [3]:
import math 
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import random as rng
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from typing import Union, List, Any
from qaoa_and_tdvp import QAOA, tdvp_optimize_qaoa, scipy_optimize, QAOAResult, Qobj
from MaxCut import MaxCut
from itertools import *

from qutip import tensor, basis, Qobj
from qutip.qip.operations import expand_operator, rz

from benchmark import (
    get_all_connected,
    get_connected_rn_graph,
    get_rn_qubo,
    select_if_connected,
    Benchmark,
)
from qutip.parallel import parallel_map, serial_map

# convert data into DataFrame

In [32]:
with open("..//benchmarks//RK45_n10_p-1-5_Delta-1.p","rb") as file:
    data: Benchmark = pickle.load(file)

scipy_results = list()
for d in data.results:
    items = d.get("tdvp").__dict__.items()
    for k,v in items:
        d[k]=v
        d["algorithm"] = "tdvp"
    d["n"] = d["_qaoa"].n
    dd = d.copy()
    items = dd.get("scipy").__dict__.items()
    for k,v in items:
        dd[k]=v
        dd["algorithm"] = "scipy"
    dd["n"] = dd["_qaoa"].n
    scipy_results.append(dd)

data.results.extend(scipy_results)

df = pd.DataFrame(data.results)
df.pop("tdvp")
df.pop("scipy")
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

df.insert(2,"algorithm",df.pop("algorithm"))
df.insert(3,"_optimal_fun_value",df.pop("_optimal_fun_value"))
df.insert(4,"_num_fun_calls",df.pop("_num_fun_calls"))
df.insert(5,"_num_steps",df.pop("_num_steps"))
df.insert(6,"_prob",df.pop("_prob"))

df_tdvp = df[df.algorithm.eq("tdvp")]
df_scipy = df[df.algorithm.eq("scipy")]

In [43]:
df["silly_function"] = df._optimal_fun_value/10

In [48]:
df.filter(items=["silly_function", "_optimal_fun_value"])

,silly_function,_optimal_fun_value
0,-0.648205,-6.482053
1,-0.552241,-5.522408
2,-0.976187,-9.761868
3,-0.917097,-9.170969
4,-0.466481,-4.664811
...,...,...
995,-2.790327,-27.903268
996,-0.459035,-4.590350
997,-0.634758,-6.347583
998,-1.249807,-12.498074
